In [241]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [242]:
debug = True

In [243]:
conn = sqlite3.connect('livs.db')
conn.row_factory = sqlite3.Row
curs = conn.cursor()



result = []
for row in curs.execute('select * from livs'):
    result.append(row)

conn.close()

dataset = np.array(result)
if debug:
    print (dataset)

[['Talg nöt' '1' 656.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86 None]
 ['Späck gris' '2' 763.0 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ['Ister gris' '3' 884.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ..., 
 ['Havredryck oberikad' '5964' 40.6 ...,
  'Vegetabiliska produkter och mjölkersättning' 46 None]
 ['Kryddblandning taco' '5973' 372.6 ..., 'Kryddor' 112 None]
 ['Tortilla wrap' '5974' 313.7 ..., 'Mjukt bröd' 12 'train']]


In [299]:
relevant_columns = [ 2,  4,  5,  6,  7,  8,  9, 10, 42, 50]
#[4,5,6]

#[ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]

In [300]:
columns = dataset.T[relevant_columns].T
columns

array([[656.3, 0.0, 71.0, ..., 0.3, 0.0, 0.3],
       [763.0, 0.0, 85.0, ..., 0.7, 0.0, 0.2],
       [884.3, 0.0, 100.0, ..., 0.0, 0.0, 0.1],
       ..., 
       [40.6, 7.6, 0.5, ..., 0.2, 0.0, 0.1],
       [372.6, 72.6, 3.55, ..., 0.0, 0.0, 11.35],
       [313.7, 57.5, 4.8, ..., 0.0, 0.0, 0.85]], dtype=object)

In [301]:
 def cluster(dataset):
    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataset)

    centroids = kmeans.cluster_centers_
    clusters = kmeans.labels_

    if debug:
        print (centroids)
        print (clusters)

    return clusters, centroids

In [302]:
numClusters = 50

In [303]:
clusters, centroids = cluster(columns)

[[  7.28906977e+01   1.26686047e+01   9.61162791e-01   1.92069767e+00
    2.66651163e+00   8.08983721e+01   1.16279070e-03   8.95581395e-01
    2.92279070e+01   7.19767442e-01]
 [  3.87818750e+02   8.08437500e+01   3.21812500e+00   5.87604167e+00
    3.67083333e+00   4.84333333e+00   5.55111512e-17   1.55208333e+00
    1.52083333e-01   2.93354167e+00]
 [  2.55983333e+02   8.96907407e+00   1.86551852e+01   1.32242593e+01
    4.25185185e-01   5.59740741e+01   1.11022302e-16   2.71037037e+00
    9.90740741e-01   1.70203704e+00]
 [  6.45630000e+02   3.23700000e+00   6.80100000e+01   5.66000000e+00
    4.22000000e+00   1.71200000e+01   2.77555756e-17   1.72300000e+00
    4.70000000e-01   1.19300000e+00]
 [  1.39133913e+02   2.37973913e+00   7.09417391e+00   1.62053043e+01
    4.19043478e-01   7.20303478e+01   1.56521739e-03   1.84443478e+00
    4.41478261e+00   2.12565217e+00]
 [  8.84300000e+02   0.00000000e+00   1.00000000e+02   1.77635684e-15
    4.44089210e-16  -2.13162821e-14   8.32667

446 [ 3 11  5 ..., 17  1 20]
447 [ 3 11  5 ..., 17  1 20]
448 [ 3 11  5 ..., 17  1 20]
449 [ 3 11  5 ..., 17  1 20]
450 [ 3 11  5 ..., 17  1 20]
451 [ 3 11  5 ..., 17  1 20]
452 [ 3 11  5 ..., 17  1 20]
453 [ 3 11  5 ..., 17  1 20]
454 [ 3 11  5 ..., 17  1 20]
455 [ 3 11  5 ..., 17  1 20]
456 [ 3 11  5 ..., 17  1 20]
457 [ 3 11  5 ..., 17  1 20]
458 [ 3 11  5 ..., 17  1 20]
459 [ 3 11  5 ..., 17  1 20]
460 [ 3 11  5 ..., 17  1 20]
461 [ 3 11  5 ..., 17  1 20]
462 [ 3 11  5 ..., 17  1 20]
463 [ 3 11  5 ..., 17  1 20]
464 [ 3 11  5 ..., 17  1 20]
465 [ 3 11  5 ..., 17  1 20]
466 [ 3 11  5 ..., 17  1 20]
467 [ 3 11  5 ..., 17  1 20]
468 [ 3 11  5 ..., 17  1 20]
469 [ 3 11  5 ..., 17  1 20]
470 [ 3 11  5 ..., 17  1 20]
471 [ 3 11  5 ..., 17  1 20]
472 [ 3 11  5 ..., 17  1 20]
473 [ 3 11  5 ..., 17  1 20]
474 [ 3 11  5 ..., 17  1 20]
475 [ 3 11  5 ..., 17  1 20]
476 [ 3 11  5 ..., 17  1 20]
477 [ 3 11  5 ..., 17  1 20]
478 [ 3 11  5 ..., 17  1 20]
479 [ 3 11  5 ..., 17  1 20]
480 [ 3 11  5 

893 [ 3 11  5 ..., 17  1 20]
894 [ 3 11  5 ..., 17  1 20]
895 [ 3 11  5 ..., 17  1 20]
896 [ 3 11  5 ..., 17  1 20]
897 [ 3 11  5 ..., 17  1 20]
898 [ 3 11  5 ..., 17  1 20]
899 [ 3 11  5 ..., 17  1 20]
900 [ 3 11  5 ..., 17  1 20]
901 [ 3 11  5 ..., 17  1 20]
902 [ 3 11  5 ..., 17  1 20]
903 [ 3 11  5 ..., 17  1 20]
904 [ 3 11  5 ..., 17  1 20]
905 [ 3 11  5 ..., 17  1 20]
906 [ 3 11  5 ..., 17  1 20]
907 [ 3 11  5 ..., 17  1 20]
908 [ 3 11  5 ..., 17  1 20]
909 [ 3 11  5 ..., 17  1 20]
910 [ 3 11  5 ..., 17  1 20]
911 [ 3 11  5 ..., 17  1 20]
912 [ 3 11  5 ..., 17  1 20]
913 [ 3 11  5 ..., 17  1 20]
914 [ 3 11  5 ..., 17  1 20]
915 [ 3 11  5 ..., 17  1 20]
916 [ 3 11  5 ..., 17  1 20]
917 [ 3 11  5 ..., 17  1 20]
918 [ 3 11  5 ..., 17  1 20]
919 [ 3 11  5 ..., 17  1 20]
920 [ 3 11  5 ..., 17  1 20]
921 [ 3 11  5 ..., 17  1 20]
922 [ 3 11  5 ..., 17  1 20]
923 [ 3 11  5 ..., 17  1 20]
924 [ 3 11  5 ..., 17  1 20]
925 [ 3 11  5 ..., 17  1 20]
926 [ 3 11  5 ..., 17  1 20]
927 [ 3 11  5 

1393 [ 3 11  5 ..., 17  1 20]
1394 [ 3 11  5 ..., 17  1 20]
1395 [ 3 11  5 ..., 17  1 20]
1396 [ 3 11  5 ..., 17  1 20]
1397 [ 3 11  5 ..., 17  1 20]
1398 [ 3 11  5 ..., 17  1 20]
1399 [ 3 11  5 ..., 17  1 20]
1400 [ 3 11  5 ..., 17  1 20]
1401 [ 3 11  5 ..., 17  1 20]
1402 [ 3 11  5 ..., 17  1 20]
1403 [ 3 11  5 ..., 17  1 20]
1404 [ 3 11  5 ..., 17  1 20]
1405 [ 3 11  5 ..., 17  1 20]
1406 [ 3 11  5 ..., 17  1 20]
1407 [ 3 11  5 ..., 17  1 20]
1408 [ 3 11  5 ..., 17  1 20]
1409 [ 3 11  5 ..., 17  1 20]
1410 [ 3 11  5 ..., 17  1 20]
1411 [ 3 11  5 ..., 17  1 20]
1412 [ 3 11  5 ..., 17  1 20]
1413 [ 3 11  5 ..., 17  1 20]
1414 [ 3 11  5 ..., 17  1 20]
1415 [ 3 11  5 ..., 17  1 20]
1416 [ 3 11  5 ..., 17  1 20]
1417 [ 3 11  5 ..., 17  1 20]
1418 [ 3 11  5 ..., 17  1 20]
1419 [ 3 11  5 ..., 17  1 20]
1420 [ 3 11  5 ..., 17  1 20]
1421 [ 3 11  5 ..., 17  1 20]
1422 [ 3 11  5 ..., 17  1 20]
1423 [ 3 11  5 ..., 17  1 20]
1424 [ 3 11  5 ..., 17  1 20]
1425 [ 3 11  5 ..., 17  1 20]
1426 [ 3 1

In [304]:
dataset.T[1]

array(['1', '2', '3', ..., '5964', '5973', '5974'], dtype=object)

In [305]:
clusters

array([ 3, 11,  5, ..., 17,  1, 20], dtype=int32)

In [306]:
unique, counts = np.unique(clusters, return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
cluster_distribution = np.array(list(zip(unique, counts)))
print(cluster_distribution, np.std(cluster_distribution.T[1]), np.mean(cluster_distribution.T[1]),np.median(cluster_distribution.T[1]),np.average(cluster_distribution.T[1]))


[[  0  43]
 [  1  48]
 [  2  54]
 [  3  10]
 [  4 115]
 [  5  16]
 [  6  20]
 [  7  29]
 [  8  59]
 [  9  42]
 [ 10  56]
 [ 11  26]
 [ 12   7]
 [ 13 115]
 [ 14   3]
 [ 15  23]
 [ 16  49]
 [ 17  90]
 [ 18  64]
 [ 19  40]
 [ 20  32]
 [ 21 136]
 [ 22   4]
 [ 23  77]
 [ 24   5]
 [ 25  25]
 [ 26  94]
 [ 27  34]
 [ 28  23]
 [ 29  20]
 [ 30  12]
 [ 31  10]
 [ 32  34]
 [ 33  49]
 [ 34 119]
 [ 35   3]
 [ 36  71]
 [ 37  14]
 [ 38  71]
 [ 39  49]
 [ 40 109]
 [ 41  14]
 [ 42  35]
 [ 43  31]
 [ 44   4]
 [ 45   8]
 [ 46   2]
 [ 47  41]
 [ 48   3]
 [ 49  50]] 34.636720399 41.76 34.0 41.76


In [310]:
ind=np.where(cluster_distribution.T[1]>40)
ind

(array([ 0,  1,  2,  4,  8,  9, 10, 13, 16, 17, 18, 21, 23, 26, 33, 34, 36,
        38, 39, 40, 47, 49]),)

In [314]:
unique, counts = np.unique(dataset.T[60], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupp_storlek = np.array(list(zip(unique, counts)))
print(huvudgrupp_storlek)   

[['' '7']
 ['Algprodukter' '1']
 ['Baljväxter (bönor, linser och ärter)' '51']
 ['Blodmat' '2']
 ['Blodprodukter o rätter' '8']
 ['Buljong' '18']
 ['Bullar kakor tårtor mm' '86']
 ['Bär färska frysta' '24']
 ['Chips popcorn o dyl' '11']
 ['Choklad' '7']
 ['Cider alkoläsk drink' '3']
 ['Deg och gräddade skal och bottnar' '3']
 ['Dessertost' '8']
 ['Dryck' '1']
 ['Efterrätter' '31']
 ['Fisk färsk fryst kokt' '68']
 ['Fisk o skaldjursprodukter o rätter' '69']
 ['Fisk rökt' '13']
 ['Fisk stekt ej panerad' '7']
 ['Flingor - frukostflingor' '28']
 ['Flytande matfettsblandning' '7']
 ['Frukt färsk fryst' '42']
 ['Frukt o bär' '2']
 ['Frukt o bär konserverade' '17']
 ['Frukt o bär torkade' '14']
 ['Frukt o nötblandningar bars' '2']
 ['Fruktjuice mm' '22']
 ['Färskost o kvarg' '15']
 ['Fågel' '39']
 ['Fågelprodukter o rätter' '5']
 ['Gelatin agar agar' '2']
 ['Glass' '25']
 ['Godis ej choklad' '11']
 ['Godis som innehåller choklad' '15']
 ['Grädde creme fraiche' '23']
 ['Grönsaker' '94']
 ['Grö

In [372]:
#a=np.array([])
for item in ind[0]:
    print("Kluster " + str(item))
    #print(dataset[np.where(clusters==item)][:,[0,60]])
    a=dataset[np.where(clusters==item)][:,[0,60]]
    #print(a[0][1])
    unique, counts = np.unique(a.T[1], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
    huvudgrupp_distribution = np.array(list(zip(unique, counts)))
    order=np.argsort(huvudgrupp_distribution.T[1].astype(int))
    order = np.flipud(order)
    #print (order)
    huvudgrupp_distribution = huvudgrupp_distribution[order]
    totals = np.array([])
    for item in huvudgrupp_distribution:
        a=np.where(huvudgrupp_storlek.T[0] == item [0])
        print(a[0], huvudgrupp_storlek[a[0],1], item[0])
        totals=np.append(totals,huvudgrupp_storlek[a[0],1])
    samlad_array = np.concatenate((huvudgrupp_distribution.T,[totals]),axis=0).T
    print(np.shape(huvudgrupp_distribution), np.shape([totals]))
    print(huvudgrupp_distribution)
    print("totals" + str(totals.T))
    print (samlad_array)
    print()
    part = lambda x: x[1] / x[2]
    med_andel = np.vstack( (samlad_array.T, np.array(map(part, samlad_array))) ).T
    print(med_andel)


Kluster 0
[74] ['31'] Potatis
[36] ['55'] Grönsaks- rotfrukts- baljväxträtter och produkter
[21] ['42'] Frukt färsk fryst
[102] ['26'] Söta soppor kräm o efterrättssås
[23] ['17'] Frukt o bär konserverade
[7] ['24'] Bär färska frysta
[2] ['51'] Baljväxter (bönor, linser och ärter)
[80] ['21'] Rotfrukter
[37] ['9'] Grönsaksblandningar med rotfrukter och eller baljväxter
[107] ['15'] Vatten mineralvatten
[82] ['10'] Sallad blandad mat
[81] ['19'] Saft läsk cider u alkohol
[55] ['80'] Köttprodukter kötträtter
(13, 2) (1, 13)
[['Potatis' '11']
 ['Grönsaks- rotfrukts- baljväxträtter och produkter' '6']
 ['Frukt färsk fryst' '6']
 ['Söta soppor kräm o efterrättssås' '3']
 ['Frukt o bär konserverade' '3']
 ['Bär färska frysta' '3']
 ['Baljväxter (bönor, linser och ärter)' '3']
 ['Rotfrukter' '2']
 ['Grönsaksblandningar med rotfrukter och eller baljväxter' '2']
 ['Vatten mineralvatten' '1']
 ['Sallad blandad mat' '1']
 ['Saft läsk cider u alkohol' '1']
 ['Köttprodukter kötträtter' '1']]
totals

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [184]:
dataset[(clusters==ind)]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2088 but corresponding boolean dimension is 1

## Läsa ner klustren i databasen

Kör inte nästa i onödan. Fyll på databasen kolumn för kolumn i stället

In [70]:
#Denna skapar tabellen kluster

conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

curs.execute('drop table if exists kluster;')

# Backslash bara för radbrytningen, obs...
sql = 'CREATE TABLE IF NOT EXISTS kluster (Livsmedelsnummer TEXT PRIMARY KEY,\
kluster_3 INT DEFAULT -1, kluster_3_3 INT DEFAULT -1, kluster_20 INT DEFAULT -1,\
kluster_20_3 INT DEFAULT -1);'


curs.execute(sql)
conn.commit()

sql2 = 'INSERT INTO kluster (Livsmedelsnummer) SELECT Livsmedelsnummer from livs;'

curs.execute(sql2)
conn.commit()

conn.close()

In [79]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

for index, Livsmedelsnummer in enumerate(dataset.T[1]):
    sql = 'UPDATE kluster SET kluster_3_3 = '+str(clusters[index])+' WHERE Livsmedelsnummer = "'+Livsmedelsnummer+'";'
    print(sql)
    curs.execute(sql)
#curs.execute('UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";')
conn.commit()
conn.close()

UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "1";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "2";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "3";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "4";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "5";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "6";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "7";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "8";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "9";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "10";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "12";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "13";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "17";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "18";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "20";
UPDATE kluster SET 

UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1236";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1237";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1238";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1239";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1240";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1241";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1242";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1243";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1244";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1245";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1246";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1247";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1248";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1249";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmed

In [80]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
   
kluster_object = []
for row in curs.execute('SELECT * from kluster WHERE kluster_3_3 != -1;'):
    kluster_object.append(row)

        
kluster = np.array(kluster_object)

print(kluster)

conn.close()

[['1' '-1' '2' '-1' '-1']
 ['2' '-1' '2' '-1' '-1']
 ['3' '-1' '2' '-1' '-1']
 ..., 
 ['5964' '-1' '0' '-1' '-1']
 ['5973' '-1' '1' '-1' '-1']
 ['5974' '-1' '1' '-1' '-1']]


In [69]:
(dataset==str(kluster.T[0]))

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

## Visualisera med Plotly

In [ ]:
# transpose to work with Plotly-format (don't remember if it is sklearn.cluster or plotly that is counterintuitive)
dataToChart = dataset.T

# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(3)] for _ in range(numClusters)]

for num, cluster in enumerate(clusters):
    if debug:
        print (str(num), str(clusters))
    segmentedData[cluster][0].append(dataToChart[0][num])
    segmentedData[cluster][1].append(dataToChart[1][num])
    segmentedData[cluster][2].append(dataToChart[2][num])

#if debug:
    #print(segmentedData)

In [69]:
#plotly.offline.init_notebook_mode(connected=True)
    
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][2],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1

layout = go.Layout(
    scene=go.Scene(
        xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
        yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
        zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
    ),
    height = 800,
    width = 800,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='table1')